In [1]:
import cv2
import numpy as np

In [2]:
def make_coordinates(lane_image, line_parameters):
    slope, intercept = line_parameters
    y1 = lane_image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])

In [3]:
def average_slope_intercept(lane_image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        elif slope > 0:
            right_fit.append((slope, intercept))
    if len(left_fit) > 0 and len(right_fit) > 0:
        left_fit_average = np.average(left_fit, axis=0)
        right_fit_average = np.average(right_fit, axis=0)
        left_line = make_coordinates(lane_image, left_fit_average)
        right_line = make_coordinates(lane_image, right_fit_average)
        return np.array([left_line, right_line])
    else:
        return None


In [4]:
def canny(lane_image):
    gray = cv2.cvtColor(lane_image,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    canny = cv2.Canny(blur,50,150)
    return canny

In [5]:
def display_lines(lane_image, lines):
    line_image = np.zeros_like(lane_image)  ##black image
    if lines is not None:
        for x1,y1,x2,y2 in lines:
            cv2.line(line_image, (x1,y1), (x2,y2), (255,0,0),10) ## coordinate, colors, and thickness which is 10.
    return line_image
            

In [6]:
def region_of_interest(lane_image):
    height = lane_image.shape[0]
    polygons = np.array([
        [(200,height),(1100,height),(550,250)]
    ])
    mask = np.zeros_like(lane_image)
    cv2.fillPoly(mask,polygons, 255) ##color white 255
    masked_image = cv2.bitwise_and(lane_image,mask)
    return masked_image

In [7]:
image = cv2.imread('test_image.jpg')

In [8]:
lane_image = np.copy(image)

In [9]:
##canny_image = canny(lane_image)
##cropped_image = region_of_interest(canny_image)
##lines = cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength = 40,maxLineGap = 5)
##averaged_lines = average_slope_intercept(lane_image, lines)
##line_image = display_lines(lane_image,averaged_lines)
##mix_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)

In [10]:
cap = cv2.VideoCapture("test2.mp4")
while(cap.isOpened()):
    _, frame = cap.read()
    canny_image = canny(frame)
    cropped_image = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength = 40,maxLineGap = 5)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame,averaged_lines)
    mix_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    cv2.imshow('result', mix_image)
    if cv2.waitKey(1) == ord('q'):  ## To exit the output screen window we have to press q
        break
    
cap.release()   ## Closing the video file
cv2.destroyAllWindows()   ## destroying the window that we were on